***GENERATED CODE FOR zillowrent PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Aberdeen', 'max': 'Zion', 'missing': '0', 'distinct': '491'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Metro', 'transformation_label': 'String Indexer'}], 'feature': 'Metro', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aberdeen', 'max': 'Yakima', 'missing': '0', 'distinct': '211'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Metro'}, {'feature_label': 'Metro', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Metro')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'County', 'transformation_label': 'String Indexer'}], 'feature': 'County', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adams', 'max': 'Yavapai', 'missing': '0', 'distinct': '290'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'County'}, {'feature_label': 'County', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('County')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WV', 'missing': '0', 'distinct': '36'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run zillowrentHooks.ipynb
try:
	#sourcePreExecutionHook()

	price = DBFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1634804423374_1609743113086_price.csv', 'filename': '1634804423374_1609743113086_price.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Real Estate Models/Zillow Rent Index - Regression/price.csv', 'viewFileName': 'price.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://52.142.62.122:4040/api/read/hdfs'}")
	#sourcePostExecutionHook(price)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run zillowrentHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(price,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "City Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "87895.09", "stddev": "127314.31", "min": "3312", "max": "399271", "missing": "0"}, "updatedLabel": "City Code"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aberdeen", "max": "Zion", "missing": "0", "distinct": "491"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Metro", "transformation_label": "String Indexer"}], "feature": "Metro", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aberdeen", "max": "Yakima", "missing": "0", "distinct": "211"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Metro"}, {"transformationsData": [{"feature_label": "County", "transformation_label": "String Indexer"}], "feature": "County", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adams", "max": "Yavapai", "missing": "0", "distinct": "290"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "County"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AL", "max": "WV", "missing": "0", "distinct": "36"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Population Rank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5974.56", "stddev": "3636.38", "min": "8", "max": "12372", "missing": "0"}, "updatedLabel": "Population Rank"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2010", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1408.0", "stddev": "737.72", "min": "624", "max": "7746", "missing": "0"}, "updatedLabel": "November 2010"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2010", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1414.72", "stddev": "749.76", "min": "636", "max": "7999", "missing": "0"}, "updatedLabel": "December 2010"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1416.66", "stddev": "745.92", "min": "633", "max": "7633", "missing": "0"}, "updatedLabel": "January 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1414.79", "stddev": "736.37", "min": "629", "max": "7754", "missing": "0"}, "updatedLabel": "February 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1409.34", "stddev": "723.56", "min": "621", "max": "7708", "missing": "0"}, "updatedLabel": "March 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1404.99", "stddev": "715.45", "min": "625", "max": "7522", "missing": "0"}, "updatedLabel": "April 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1401.62", "stddev": "709.65", "min": "628", "max": "7387", "missing": "0"}, "updatedLabel": "May 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1400.77", "stddev": "705.06", "min": "629", "max": "7377", "missing": "0"}, "updatedLabel": "June 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1399.31", "stddev": "699.65", "min": "622", "max": "7363", "missing": "0"}, "updatedLabel": "July 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1397.68", "stddev": "696.34", "min": "608", "max": "7356", "missing": "0"}, "updatedLabel": "August 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1395.27", "stddev": "694.4", "min": "600", "max": "7360", "missing": "0"}, "updatedLabel": "September 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1393.1", "stddev": "696.05", "min": "600", "max": "7424", "missing": "0"}, "updatedLabel": "October 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1392.97", "stddev": "702.78", "min": "610", "max": "7513", "missing": "0"}, "updatedLabel": "November 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2011", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1390.26", "stddev": "706.07", "min": "611", "max": "7582", "missing": "0"}, "updatedLabel": "December 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1389.57", "stddev": "710.04", "min": "613", "max": "7708", "missing": "0"}, "updatedLabel": "January 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1390.74", "stddev": "712.06", "min": "611", "max": "7702", "missing": "0"}, "updatedLabel": "February 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1393.21", "stddev": "712.09", "min": "607", "max": "7652", "missing": "0"}, "updatedLabel": "March 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1395.61", "stddev": "712.88", "min": "597", "max": "7675", "missing": "0"}, "updatedLabel": "April 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1397.86", "stddev": "716.21", "min": "590", "max": "7810", "missing": "0"}, "updatedLabel": "May 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1399.76", "stddev": "719.77", "min": "586", "max": "7908", "missing": "0"}, "updatedLabel": "June 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1401.35", "stddev": "721.22", "min": "587", "max": "7893", "missing": "0"}, "updatedLabel": "July 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1402.35", "stddev": "722.15", "min": "592", "max": "7859", "missing": "0"}, "updatedLabel": "August 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1402.79", "stddev": "721.14", "min": "596", "max": "7805", "missing": "0"}, "updatedLabel": "September 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1403.42", "stddev": "720.82", "min": "596", "max": "7827", "missing": "0"}, "updatedLabel": "October 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1404.93", "stddev": "719.83", "min": "592", "max": "7824", "missing": "0"}, "updatedLabel": "November 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2012", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1405.46", "stddev": "717.19", "min": "583", "max": "7828", "missing": "0"}, "updatedLabel": "December 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1407.54", "stddev": "718.68", "min": "581", "max": "7964", "missing": "0"}, "updatedLabel": "January 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1410.95", "stddev": "724.94", "min": "584", "max": "8032", "missing": "0"}, "updatedLabel": "February 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1414.67", "stddev": "732.96", "min": "592", "max": "8095", "missing": "0"}, "updatedLabel": "March 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1417.71", "stddev": "739.27", "min": "602", "max": "8124", "missing": "0"}, "updatedLabel": "April 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1420.83", "stddev": "744.87", "min": "617", "max": "8279", "missing": "0"}, "updatedLabel": "May 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1423.67", "stddev": "749.4", "min": "631", "max": "8424", "missing": "0"}, "updatedLabel": "June 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1425.9", "stddev": "752.68", "min": "639", "max": "8563", "missing": "0"}, "updatedLabel": "July 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1428.21", "stddev": "754.47", "min": "632", "max": "8686", "missing": "0"}, "updatedLabel": "August 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1432.15", "stddev": "757.16", "min": "614", "max": "8788", "missing": "0"}, "updatedLabel": "September 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1438.61", "stddev": "761.25", "min": "604", "max": "8856", "missing": "0"}, "updatedLabel": "October 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1446.5", "stddev": "766.66", "min": "603", "max": "8830", "missing": "0"}, "updatedLabel": "November 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2013", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1450.56", "stddev": "767.96", "min": "614", "max": "8744", "missing": "0"}, "updatedLabel": "December 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1452.44", "stddev": "772.34", "min": "630", "max": "8830", "missing": "0"}, "updatedLabel": "January 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1453.41", "stddev": "777.57", "min": "635", "max": "8913", "missing": "0"}, "updatedLabel": "February 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1453.38", "stddev": "783.04", "min": "634", "max": "8990", "missing": "0"}, "updatedLabel": "March 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1453.46", "stddev": "790.75", "min": "635", "max": "9070", "missing": "0"}, "updatedLabel": "April 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1453.31", "stddev": "801.65", "min": "642", "max": "9262", "missing": "0"}, "updatedLabel": "May 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1453.35", "stddev": "812.02", "min": "650", "max": "9425", "missing": "0"}, "updatedLabel": "June 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1457.8", "stddev": "826.03", "min": "649", "max": "9747", "missing": "0"}, "updatedLabel": "July 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1466.6", "stddev": "841.47", "min": "650", "max": "10111", "missing": "0"}, "updatedLabel": "August 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1479.68", "stddev": "858.33", "min": "654", "max": "10463", "missing": "0"}, "updatedLabel": "September 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1490.04", "stddev": "867.97", "min": "650", "max": "10601", "missing": "0"}, "updatedLabel": "October 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1498.97", "stddev": "875.27", "min": "655", "max": "10697", "missing": "0"}, "updatedLabel": "November 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2014", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1504.28", "stddev": "876.92", "min": "659", "max": "10727", "missing": "0"}, "updatedLabel": "December 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1508.53", "stddev": "879.39", "min": "666", "max": "10791", "missing": "0"}, "updatedLabel": "January 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1512.75", "stddev": "882.33", "min": "658", "max": "10760", "missing": "0"}, "updatedLabel": "February 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1517.26", "stddev": "886.52", "min": "656", "max": "10751", "missing": "0"}, "updatedLabel": "March 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1523.49", "stddev": "892.0", "min": "648", "max": "10775", "missing": "0"}, "updatedLabel": "April 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1530.39", "stddev": "900.19", "min": "644", "max": "10944", "missing": "0"}, "updatedLabel": "May 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1538.26", "stddev": "907.45", "min": "645", "max": "11021", "missing": "0"}, "updatedLabel": "June 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1545.4", "stddev": "912.77", "min": "657", "max": "11036", "missing": "0"}, "updatedLabel": "July 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1549.84", "stddev": "916.7", "min": "670", "max": "11043", "missing": "0"}, "updatedLabel": "August 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1550.13", "stddev": "920.25", "min": "671", "max": "11055", "missing": "0"}, "updatedLabel": "September 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1548.05", "stddev": "924.04", "min": "671", "max": "11092", "missing": "0"}, "updatedLabel": "October 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1547.57", "stddev": "928.73", "min": "667", "max": "11081", "missing": "0"}, "updatedLabel": "November 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2015", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1547.07", "stddev": "930.25", "min": "660", "max": "11012", "missing": "0"}, "updatedLabel": "December 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1550.94", "stddev": "933.3", "min": "658", "max": "10951", "missing": "0"}, "updatedLabel": "January 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1556.31", "stddev": "947.51", "min": "657", "max": "11592", "missing": "0"}, "updatedLabel": "February 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1563.99", "stddev": "956.47", "min": "648", "max": "11871", "missing": "0"}, "updatedLabel": "March 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1569.25", "stddev": "958.46", "min": "668", "max": "11996", "missing": "0"}, "updatedLabel": "April 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1573.06", "stddev": "948.47", "min": "669", "max": "11452", "missing": "0"}, "updatedLabel": "May 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1574.46", "stddev": "940.96", "min": "654", "max": "11117", "missing": "0"}, "updatedLabel": "June 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1572.77", "stddev": "935.97", "min": "638", "max": "10837", "missing": "0"}, "updatedLabel": "July 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1569.6", "stddev": "931.58", "min": "634", "max": "10551", "missing": "0"}, "updatedLabel": "August 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1566.6", "stddev": "927.48", "min": "635", "max": "10341", "missing": "0"}, "updatedLabel": "September 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1566.3", "stddev": "925.61", "min": "642", "max": "10266", "missing": "0"}, "updatedLabel": "October 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1568.58", "stddev": "927.85", "min": "655", "max": "10341", "missing": "0"}, "updatedLabel": "November 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2016", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1569.74", "stddev": "928.92", "min": "664", "max": "10434", "missing": "0"}, "updatedLabel": "December 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2017", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1570.45", "stddev": "933.32", "min": "676", "max": "10626", "missing": "0"}, "updatedLabel": "January 2017"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run zillowrentHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["City Code", "Population Rank", "November 2010", "December 2010", "January 2011", "February 2011", "March 2011", "April 2011", "May 2011", "June 2011", "July 2011", "August 2011", "September 2011", "October 2011", "November 2011", "December 2011", "January 2012", "February 2012", "March 2012", "April 2012", "May 2012", "June 2012", "July 2012", "August 2012", "September 2012", "October 2012", "November 2012", "December 2012", "January 2013", "February 2013", "March 2013", "April 2013", "May 2013", "June 2013", "July 2013", "August 2013", "September 2013", "October 2013", "November 2013", "December 2013", "January 2014", "February 2014", "March 2014", "April 2014", "May 2014", "June 2014", "July 2014", "August 2014", "September 2014", "October 2014", "November 2014", "December 2014", "January 2015", "February 2015", "March 2015", "April 2015", "May 2015", "June 2015", "July 2015", "August 2015", "September 2015", "October 2015", "November 2015", "December 2015", "January 2016", "February 2016", "March 2016", "April 2016", "May 2016", "June 2016", "July 2016", "August 2016", "September 2016", "October 2016", "November 2016", "December 2016", "January 2017", "City_stringindexer", "Metro_stringindexer", "County_stringindexer"], "State_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

